In [1]:
# dependencies
import numpy as np
import matplotlib.pyplot as plt

## Canonical ensemble:1D harmonic oscillator case

In our last session we focused in the microcanical ensemble and its state function: the entropy. Today we will explore the features of the canonical ensemble where the number of particles (N), the pressure (P) and the temperature (T) remain as constant. 

Starting with the 1-D harmonic oscillator case we have the Hamiltonian is :
$$
H = \frac{p^2}{2m} + \frac{1}{2}m \omega^2 x^2 , \\ \omega = \sqrt{\frac{k}{m}}
$$

The main difference with the microcanonical ensemble is the contact with a thermal bath of temperature T. In fact, this allow to the system to remain close to a specific temperature, thus, varying its total energy. 
Similar to number of microstates $\Omega(E)$ computed in the NVE ensemble, the canonical ensemble relies on the partition function:


$$Z(\beta)=\frac{1}{h}\int_{-\infty}^{\infty}d\textbf{p}\int_{-\infty}^{\infty}d\textbf{x}e^{-\beta H(x,p)}, \\ 
\\ \beta = \frac{1}{k_{B}T} $$

if we separate the momentum integrals from the position integrals we get:
$$Z(\beta)=\frac{1}{h}\left ( \int_{-\infty}^{\infty}d\textbf{p}e^{-\beta \textbf{p}^{2}/ 2m}\right ) \left ( \int_{-\infty}^{\infty}d\textbf{x}e^{-\frac{1}{2}\beta m \omega^{2}\textbf{x}^{2}}\right )$$

where due to the kinetic energy is always $\frac{p^2}{2m}$ we can solve the momentum integral as (considering gaussian integration):

$$ \int_{-\infty}^{\infty}d\textbf{p}e^{-\beta \textbf{p}^{2}/ 2m} = \sqrt{\frac{2 \pi m}{\beta}}$$

On the other hand, due we know the position integral and it has analytic solution for the harmonic oscillator we obtain:

$$ \int_{-\infty}^{\infty}d\textbf{x}e^{\frac{1}{2}-\beta m \omega^{2} \textbf{x}^{2}} = \sqrt{\frac{2 \pi }{\beta m \omega^{2}}}$$

Thus, the partition function can be explicitely formulated as:

$$Z(\beta) = \frac{1}{h}\frac{2 \pi}{\beta \omega}$$

The partition function is related with the Helmholtz energy trough the following relationship:

$$F = -k_{B}T\ln {Z} = -k_{B}T\ln {\left ( \frac{1}{h}\frac{2 \pi}{\beta \omega} \right )}$$